# Problem 1: Drug Trial

In this problem, you'll analyze some data from a medical drug trial. There are three exercises worth a total of ten points.

Two of the exercises allow you to use **either** Pandas **or** SQL to solve it. Choose the method that feels is more natural to you.

> **Note**: Because Vocareum only supports the `sqlite3` module with Python 3.5, this problem is tied to that version (rather than 3.6). As such, some care may be needed to ensure that you are only using features available in the corresponding versions of Python and supporting libraries (e.g., `pandas`, `sqlite3`).

## Setup

Run the following few code cells, which will load the modules and sample data you'll need for this problem.

In [256]:
import sys
import pandas as pd
import numpy as np
import sqlite3 as db

print("Python version: {}".format(sys.version))
print("pandas version: {}".format(pd.__version__))
print("numpy version: {}".format(np.__version__))
print("sqlite3 version: {}".format(db.version))

from IPython.display import display
from cse6040utils import canonicalize_tibble, tibbles_are_equivalent

Python version: 3.5.2 (default, Nov 23 2017, 16:37:01) 
[GCC 5.4.0 20160609]
pandas version: 0.23.4
numpy version: 1.15.3
sqlite3 version: 2.6.0


In [257]:
from cse6040utils import download_all

datasets = {'Drugs_soln.csv': '6df060bde8dea48986dc12650a4fbef5',
            'avg_dose_soln.csv': 'f604e3da488d489792fec220ada738f8',
            'drugs.csv': '33bb1fa5068069a483a6e05fafde40d0',
            'nst_count_soln.csv': '7519ad4764eb238a9120fa7cd1f006de',
            'nst_count_soln--corrected.csv': '81f801cd20775a51f92a1b28593c0915',
            'swt_count_soln.csv': 'fbbb7368d31856665c3e5e1b19d93d65'}

DATA_SUFFIX = "drug-trials/"
data_paths = download_all(datasets, local_suffix=DATA_SUFFIX, url_suffix=DATA_SUFFIX)
    
print("\n(All data appears to be ready.)")

'drugs.csv' is ready!
'Drugs_soln.csv' is ready!
'nst_count_soln--corrected.csv' is ready!
'swt_count_soln.csv' is ready!
'avg_dose_soln.csv' is ready!
'nst_count_soln.csv' is ready!

(All data appears to be ready.)


## The data

Company XYZ currently uses Medication A to treat all its patients and is considering a switch to Medication B. A critical part of the evaluation of Medication B is how much of it would be used among XYZ’s patients.

The company did a trial of Medication B. The data in the accompanying CSV file, `Drugs.csv`, is data taken from roughly 130 patients at least 2 months before switching medications and up to 3 months while on the new medication.

A patient can be taking medication A or medication B, but cannot be taking both at the same time.

The following code cell will read this data and store it in a dataframe named `Drugs`.

In [258]:
Drugs = pd.read_csv(data_paths['drugs.csv'], header=0)
assert len(Drugs) == 2022
Drugs.head()

,ID,Med,Admin Date,Units
0,1,Med A,7/2/12,"1,500.00"
1,1,Med A,7/6/12,"1,500.00"
2,1,Med A,7/9/12,"1,500.00"
3,1,Med A,7/11/12,"1,500.00"
4,1,Med A,7/13/12,"1,500.00"


Each row indicates that a patient (identified by his or her `'ID'`) took one **dose** of a particular drug on a particular day. The size of the dose was `'Units'`.

## Exercises

**Exercise 0** (1 points). All you have to do is read the code in the following code cell and run it. You should observe the following.

First, the `'Med'`, `'Admin Date'`, and `'Units'` columns are stored as strings initially.

Secondly, there are some handy functions in Pandas to change the `'Admin Date'` and '`Units`' columns into more "natural" native Python types, namely, a floating-point type and a Python `datetime` type, respectively. Indeed, once in this form, it is easy to use Pandas to, say, extract the month as its own column.

In [259]:
# Observe types:
for col in ['Med', 'Admin Date', 'Units']:
    print("Column '{}' has type {}.".format(col, type(Drugs[col].iloc[0])))
    
# Convert strings to "natural" types:
Drugs = pd.read_csv(data_paths['drugs.csv'], header=0)
Drugs['Units'] = pd.to_numeric(Drugs['Units'].str.replace(',',''), errors='coerce')
Drugs['Admin Date'] = pd.to_datetime(Drugs['Admin Date'], infer_datetime_format=True)
Drugs['Month'] = Drugs['Admin Date'].dt.month

print ("\nFive random records from the `Drugs` table:")
display(Drugs.iloc[np.random.choice (len (Drugs), 5)])

assert Drugs['Units'].dtype == 'float64'
assert Drugs['Month'].dtype == 'int64'

Column 'Med' has type <class 'str'>.
Column 'Admin Date' has type <class 'str'>.
Column 'Units' has type <class 'str'>.

Five random records from the `Drugs` table:


,ID,Med,Admin Date,Units,Month
1055,70,Med A,2012-07-11,9500.0,7
77,4,Med A,2012-08-27,1300.0,8
11,1,Med A,2012-08-01,1700.0,8
1138,78,Med A,2012-08-10,300.0,8
365,21,Med A,2012-07-23,2500.0,7


**Exercise 1** (1 point). Again, all you need to do is read and run the following code cell. It creates an SQLite database file named `drug_trial.db` and copies the Pandas dataframe from above into it as a table named `Drugs`.

The `conn` variable holds a live connection to this data.

In [260]:
# Import Drugs_soln dataframe above to sqlite database
# Connect to a database (or create one if it doesn't exist)
conn = db.connect('drug_trial.db')
Drugs.to_sql('Drugs', conn, if_exists='replace', index=False)
pd.read_sql_query('SELECT * FROM Drugs LIMIT 5', conn)

/usr/local/lib/python3.5/dist-packages/pandas/core/generic.py:2130: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  dtype=dtype)


,ID,Med,Admin Date,Units,Month
0,1,Med A,2012-07-02 00:00:00,1500.0,7
1,1,Med A,2012-07-06 00:00:00,1500.0,7
2,1,Med A,2012-07-09 00:00:00,1500.0,7
3,1,Med A,2012-07-11 00:00:00,1500.0,7
4,1,Med A,2012-07-13 00:00:00,1500.0,7


**Exercise 2** (2 points). **Suppose you want to know the average dose, for each medication (A and B) and month ranging from July to November.**

For example, it will turn out that in July the average dose of drug A was 5,129.56 units (rounded to two decimal places), and in September the average dose of drug B was 7.04.

Write some code to perform this calculation. Store your results in a Pandas data frame named `avg_dose` having the following three columns:
- `'Month'`: The month;
- `'Med'`: The medication, either `'Med A'` and `'Med B'`;
- `'Units'`: The average dose, **rounded to 2 decimal digits**.

> You can write either Pandas code or SQL code. If using Pandas, the data exists in the `Drugs` dataframe; if using SQL, the `conn` database connection holds a table named `Drugs`.

In [261]:
avg_dose = Drugs.copy().drop(['ID', 'Admin Date'], axis=1)
avg_dose = avg_dose[avg_dose['Month'] != 12]
avg_dose = avg_dose.groupby(['Med', 'Month']).mean().round(2).reset_index()



# Show your solution:
display(avg_dose)


,Med,Month,Units
0,Med A,7,5129.56
1,Med A,8,5645.78
2,Med A,9,5311.88
3,Med A,11,10757.14
4,Med B,9,7.04
5,Med B,10,5.78
6,Med B,11,5.60


In [262]:
# Test code
# Read what we believe is the exact result (up to permutations)
avg_dose_soln = pd.read_csv(data_paths['avg_dose_soln.csv'])

# Check that we got a data frame of the expected shape:
assert 'avg_dose' in globals(), "You need to store your results in a dataframe named `avg_dose`."
assert type(avg_dose) is type(pd.DataFrame()), "`avg_dose` does not appear to be a Pandas dataframe."
assert len(avg_dose) == len(avg_dose_soln), "The number of rows of `avg_dose` does not match our solution."
assert set(avg_dose.columns) == set(['Month', 'Med', 'Units']), "Your table does not have the right set of columns."

assert tibbles_are_equivalent(avg_dose, avg_dose_soln)
print("\n(Passed!)")


(Passed!)


**Exercise 3** (6 points). For each month, write some code to calculate the following:
- (3 points) How many patients switched from medication A to medication B? Store youre results in a Pandas dataframe named `swt_count`.
- (3 points) How many patients started on medication B, having never been on medication A before? Store your results in a Pandas dataframe named `nst_count`.

The two dataframes should have two columns: `Month` and `Count`. Again, you can choose to use SQL queries or Pandas directly to generate these dataframes.

> If it's helpful, recall that patients can only be switched from medication A to medication B, but not from B back to A.

In [263]:
# Pandas solution
def calc_who_switched(Drugs):
    who_switched = Drugs.groupby(['ID'])['Med'].unique().to_frame(name='Unique Med')
    who_switched['Med A->B?'] = who_switched.apply(lambda x: 1 if len(x['Unique Med'])==2 else 0,axis=1)
    who_switched['Newly_started'] = who_switched.apply(lambda x: 1 if x['Unique Med'].tolist()==['Med B'] else 0,axis=1)
    Drugs_with_flags = Drugs.join(who_switched, on='ID', how='left')
    return Drugs_with_flags

def solve_swt_count_using_Pandas(Drugs):
    Drugs_with_flags = calc_who_switched(Drugs)
    month_switched = Drugs_with_flags[(Drugs_with_flags['Med A->B?']==1)&(Drugs_with_flags['Med']=='Med B')]\
                        .groupby(['ID']).first()
    swt_count = month_switched.groupby(['Month'])['Med'].count().to_frame(name='Count').reset_index()
    return swt_count

swt_count = solve_swt_count_using_Pandas(Drugs)

len(swt_count_soln)
len(solve_swt_count_using_Pandas(Drugs))



len (swt_count) == len (swt_count_soln)




True

In [264]:
# Test code for exercise_a
# Read what we believe is the exact result
swt_count_soln = pd.read_csv(data_paths['swt_count_soln.csv'])

# Check that we got a data frame of the expected shape:
assert 'swt_count' in globals ()
assert type (swt_count) is type (pd.DataFrame ())
assert len (swt_count) == len (swt_count_soln)
assert set (swt_count.columns) == set (['Month', 'Count'])

print ("Number of patients who switched from Med A to Med B each month:")
display (swt_count)

assert tibbles_are_equivalent (swt_count, swt_count_soln)
print ("\n(Passed!)")

Number of patients who switched from Med A to Med B each month:


,Month,Count
0,9,71
1,10,10



(Passed!)


In [265]:
# Write your solution to compute `nst_count` in this code cell.

def solve_nst_count_with_Pandas(Drugs):
    Drugs_with_flags = calc_who_switched(Drugs)
    patients_newly_started_medB = Drugs_with_flags[Drugs_with_flags['Newly_started']==1]
    nst_count = patients_newly_started_medB.groupby(['ID']).first().groupby(['Month']).count()\
                                            .reset_index()[['Month', 'Med']].rename(columns={'Med': 'Count'})
    return nst_count

nst_count = solve_nst_count_with_Pandas(Drugs)

In [266]:
# Test code for exercise_b
# Read what we believe is the exact result
nst_count_soln_corrected = pd.read_csv(data_paths['nst_count_soln--corrected.csv'])
nst_count_soln_ok = pd.read_csv(data_paths['nst_count_soln.csv'])

# Check that we got a data frame of the expected shape:
assert 'nst_count' in globals ()
assert type (nst_count) is type (pd.DataFrame ())
assert (len (nst_count) == len (nst_count_soln_corrected)) or (len (nst_count) == len (nst_count_soln_ok))
assert set (nst_count.columns) == set (['Month', 'Count'])

print ("Number of patients who newly start Med B each month:")
display (nst_count)

assert tibbles_are_equivalent(nst_count, nst_count_soln_ok) \
       or tibbles_are_equivalent(nst_count, nst_count_soln_corrected)
print ("\n(Passed!)")

Number of patients who newly start Med B each month:


,Month,Count
0,9,5
1,10,5
2,11,6



(Passed!)


In [267]:
# Some cleanup code
conn.close()

**Fin!** Well done! If you have successfully completed this problem, move on to the next one. Good luck!